In [1]:
import pandas as pd
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
file_name = "strucutral_hp_results.xlsx"

In [3]:
data = pd.read_excel(file_name, index_col=0)
data = data.round(4)

In [4]:
data.sort_values(by="Average of Metrics", inplace=True, ascending=False)
data.head(5)

,Loss Function,Network,Input Type,Num of params to predict,Uses Activation,Sum of Metrics,Average of Metrics,Expectation MSE
12,combined_loss,encoder_with_MLP,pulse_parameters,9,True,2.4580,0.8193,0.0223
35,expectation_loss,encoder_with_MLP,pulse_parameters,9,True,2.4571,0.8190,0.0229
43,trace_distance_loss,encoder_with_MLP,pulse_parameters,9,True,2.4547,0.8182,0.0222
85,trace_distance_loss,simple_ANN,time_series,9,True,2.4342,0.8114,0.0206
73,parameter_loss,rnn_with_MLP,pulse_parameters,9,True,2.4330,0.8110,0.0303


In [5]:
mean_metric = round(data["Average of Metrics"].mean(), 4)
std_metric = round(data["Average of Metrics"].std(), 4)
print(f"Average of Metrics Mean: {mean_metric} +/- {std_metric}")

Average of Metrics Mean: 0.7214 +/- 0.0678


In [6]:
mean_expectation_mse = round(data["Expectation MSE"].mean(), 4)
std_expectation_mse = round(data["Expectation MSE"].std(), 4)
print(f"Expectation MSE Mean: {mean_expectation_mse} +/- {std_expectation_mse}")

Expectation MSE Mean: 0.0774 +/- 0.1274


In [7]:
def compute_mean_std_table(data: pd.DataFrame, column: str) -> pd.DataFrame:
    return round(
        data.groupby(column)
        .agg(
            {
                "Average of Metrics": ["mean", "std"],
                "Expectation MSE": ["mean", "std"],
            }
        )
        .rename_axis(column),
        4,
    )


def compute_f_statistic_p_value(
    data: pd.DataFrame, column: str
) -> pd.DataFrame:
    grouped_data = data.groupby(column)

    group_values = [
        group["Average of Metrics"].values for _, group in grouped_data
    ]

    f_statistic, p_value = f_oneway(*group_values)

    return round(f_statistic, 4), round(p_value, 4)

In [8]:
display(compute_mean_std_table(data, "Loss Function"))

f_statistic, p_value = compute_f_statistic_p_value(data, "Loss Function")

print(f"F-statistic: {f_statistic}, P-value: {p_value}") 



posthoc = pairwise_tukeyhsd(data['Average of Metrics'], data['Loss Function'], alpha=0.05)

print(posthoc)


Average of Metrics         Expectation MSE        
                                  mean     std            mean     std
Loss Function                                                         
combined_loss                   0.7328  0.0567          0.0213  0.0003
expectation_loss                0.7361  0.0519          0.0212  0.0006
parameter_loss                  0.6772  0.0860          0.2459  0.1661
trace_distance_loss             0.7394  0.0543          0.0213  0.0004

F-statistic: 5.1513, P-value: 0.0025
            Multiple Comparison of Means - Tukey HSD, FWER=0.05            
     group1             group2       meandiff p-adj   lower   upper  reject
---------------------------------------------------------------------------
   combined_loss    expectation_loss   0.0033 0.9979 -0.0448  0.0515  False
   combined_loss      parameter_loss  -0.0555 0.0172 -0.1037 -0.0073   True
   combined_loss trace_distance_loss   0.0066  0.984 -0.0415  0.0548  False
expectation_loss      parameter_loss  -0.0588 0.0101  -0.107 -0.0107   True
expectation_loss trace_distance_loss   0.0033  0.998 -0.0449  0.0514  False
  parameter_loss trace_distance_loss   0.0621 0.0059   0.014  0.1103   True
---------------------------------------------------------------------------


In [9]:
display(compute_mean_std_table(data, "Network"))

f_statistic, p_value = compute_f_statistic_p_value(data, "Network")

print(f"F-statistic: {f_statistic}, P-value: {p_value}") 


Average of Metrics         Expectation MSE        
                               mean     std            mean     std
Network                                                            
encoder_with_MLP             0.7210  0.0681          0.0714  0.1154
rnn_with_MLP                 0.7259  0.0647          0.0711  0.1150
simple_ANN                   0.7172  0.0723          0.0898  0.1516

F-statistic: 0.1282, P-value: 0.8798


In [10]:
display(compute_mean_std_table(data, "Input Type"))

f_statistic, p_value = compute_f_statistic_p_value(data, "Input Type")

print(f"F-statistic: {f_statistic}, P-value: {p_value}") 

Average of Metrics         Expectation MSE        
                               mean     std            mean     std
Input Type                                                         
pulse_parameters             0.7255  0.0637          0.0809  0.1233
time_series                  0.7172  0.0721          0.0739  0.1327

F-statistic: 0.3568, P-value: 0.5517


In [11]:
display(compute_mean_std_table(data, "Num of params to predict"))

f_statistic, p_value = compute_f_statistic_p_value(data, "Num of params to predict")

print(f"F-statistic: {f_statistic}, P-value: {p_value}") 

Average of Metrics         Expectation MSE        
                                       mean     std            mean     std
Num of params to predict                                                   
9                                    0.7591  0.0529          0.0679  0.1105
12                                   0.6837  0.0600          0.0870  0.1430

F-statistic: 42.682, P-value: 0.0


In [12]:
display(compute_mean_std_table(data, "Uses Activation"))

f_statistic, p_value = compute_f_statistic_p_value(data, "Uses Activation")

print(f"F-statistic: {f_statistic}, P-value: {p_value}") 

Average of Metrics         Expectation MSE        
                              mean     std            mean     std
Uses Activation                                                   
False                       0.6821  0.0574          0.1051  0.1527
True                        0.7607  0.0534          0.0498  0.0891

F-statistic: 48.2831, P-value: 0.0
